In [82]:
import glob

import tensorflow as tf

from tensorflow.python.summary.summary_iterator import summary_iterator
from tensorflow.python.framework import tensor_util

from matplotlib import pyplot as plt
import yaml

import itertools

import numpy as np

import pandas as pd

import shutil

In [80]:
def read_model_metrics(model_path):
    with open(f"{model_path}/summary.yml", "r") as fh:
        summary = yaml.safe_load(fh)

    assert summary["metrics"]["train"]["loss"] > 0

    summary[f"metrics:test:accuracy_L12"] = summary["metrics"]["test"]["accuracy_L12"]
    del summary["metrics"]    
    summary["path"] = model_path
    summary['name'] = model_path.split("/")[-1]
    return summary

read_model_metrics(
    "../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-25--17-46-49",
)

{'M': 1,
 'beta': 0.001,
 'dataset': 'mnist',
 'epoch': 200,
 'lr': 0.0001,
 'model': 'vdb/e1:1024|e2:1024|z:256',
 'strategy': 'seq/d:5|e:1',
 'metrics:test:accuracy_L12': 0.9843003153800964,
 'path': '../artifacts-server/mnist--various-opts-M1/vdb-mnist--2019-11-25--17-46-49',
 'name': 'vdb-mnist--2019-11-25--17-46-49'}

In [41]:
def get_models_from_dirs(dirs):
    files = []
    for d in dirs:
        files.extend(glob.glob(f"{d}/*/summary.yml"))
    
    files = list(map(lambda f: "/".join(f.split("/")[:-1]), files))
    print(f"Reading {','.join(dirs)}")
    print(f"we have {len(files)} models")
    return files

def read_summary_from_dir(dirs):
    results = []

    dirs = glob.glob(f"{dirs}/summary.yml")

    for d in dirs:
        d = d.replace("/summary.yml", "")
        results.append(read_model_metrics(d))
    return pd.DataFrame(results)

In [61]:
df = read_summary_from_dir("../artifacts-server/mpg-server/experiment-robustness-cifar10-beta05-vib/*M6/*")

df.sort_values(by="metrics:test:accuracy_L12", ascending=False)

,M,batch_size,beta,class_loss,cov_type,dataset,epoch,lr,metrics:test:accuracy_L12,model,name,strategy
0,6,100,0.00001,vib,diag,cifar10,50,0.01,0.8266,resnet20/z:20,resnet20-vib-diag-cifar10--2019-12-27--19-08-2...,oneshot
2,6,100,0.00001,vib,diag,cifar10,50,0.01,0.8165,resnet20/z:20,resnet20-vib-diag-cifar10--2019-12-27--18-42-0...,oneshot
1,6,100,0.00001,vib,diag,cifar10,50,0.01,0.8145,resnet20/z:20,resnet20-vib-diag-cifar10--2019-12-27--19-16-3...,oneshot
3,6,100,0.00001,vib,diag,cifar10,50,0.01,0.8102,resnet20/z:20,resnet20-vib-diag-cifar10--2019-12-27--18-55-0...,oneshot


In [94]:

for b in [3, 4, 5]:
    for m in [1, 6, 12]:
        df = read_summary_from_dir(f"../artifacts-server/mpg-server/experiment-robustness-cifar10-beta0{b}-vib/*M{m}/*")
        if len(df) == 0:
            continue
            
        df = df.sort_values(by="metrics:test:accuracy_L12", ascending=False)
        for strategy in df.strategy.unique():

            exp = df[df.strategy == strategy].iloc[0, :]
            if not exp.empty:
                continue
            assert b == -np.log10(exp["beta"])
            assert m == exp["M"]

            filename = "beta0%d-M%d-%s-%s" % (b, m, exp["class_loss"], exp["strategy"].replace("/k:", "k"))
            dest_path = f"./figures/cifar10-latent-representations/{filename}.png"
            print("copying", exp["name"], "->", dest_path)
            shutil.copy(exp["path"]+"/umap-embedding.png", dest_path)